# This notebook experiments with the combination of consensus in estimation and using FIM mixture for sensor location update.

**Also, we only consider fixed-topology networks in the experiments below.**

In [13]:

%matplotlib inline
import numpy as np
from scipy import special
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

import pickle as pkl
import networkx as nx
from matplotlib import style
import jax.numpy as jnp
from jax import jacfwd, jit,grad
from functools import partial


from Robot import Robot
from virtual_sensor import virtual_sensor
from utils.DynamicFilters import getDynamicFilter,joint_meas_func
from utils.FIMPathPlanning import FIM_ascent_path_planning 
from utils.dLdp import FIM,FIM_mix,analytic_dhdz,analytic_dLdp
from utils.regions import CircleExterior
from tracking_log import logger



%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
class ConsensusEKF:
    """
        The distributed Extended Kalman Filter with consensus on qhat enabled.
        
        The update formula is modified from equation (20) in
        
            Olfati-Saber, R. (2009). Kalman-Consensus Filter: Optimality, stability, and performance. Proceedings of the 48h IEEE Conference on Decision and Control (CDC) Held Jointly with 2009 28th Chinese Control Conference, 7036–7042. https://doi.org/10.1109/CDC.2009.5399678
        
        The main idea is to perform consensus on z(mean estimation) in the correction step, but do not consensus on covariant matrices to reduce the computation/communication overhead.
        
        We only communicate z after prediction step, so the consensus term is based on z predicted instead of z corrected, which is a subtle difference from Olfati-Saber's paper.
        
        The estimator assumes a constant-velocity source movement model, with the velocity to be the fundamental uncertainty.
    """
    def __init__(self,q_0,R_mag=1,Q_mag=1,C_gain = 0.1):
        '''
        q_0 should be a vector, the initial guess of source position.
        
        R_mag,Q_mag,C_gain should be scalars.
        '''
        
        self.q = q_0 # Mean
        self.qdot = np.zeros(self.q.shape) # Velocity Mean
        
        self.z = np.hstack([self.q,self.qdot]) # The source predicted state, public variable.
        self._zbar = np.hstack([self.q,self.qdot]) # The source corrected state, private variable.
        
        
        
        self.qdim = len(self.q)
        
        self.P = np.eye(len(self.z)) # Covariance of [q,qdot]. Initialized to be the identity matrix
        
        self.R_mag = R_mag 
        self.Q_mag = Q_mag
        self.C_gain = C_gain # The consensus gain. See the update function.
    
    def dfdz(self,z):
        n = len(z)//2
        O =np.zeros((n,n))
        I=np.eye(n)
        return np.vstack([np.hstack([I,I]),np.hstack([O,I])])

    def f(self,z):
        """
            The constant velocity model.
        """
        A = self.dfdz(z)
        
        return A.dot(z)
    def update(self,h,dhdz,y,p,z_neighbor):
        """
        h is a function handle h(z,p), the measurement function that maps z,p to y.
        dhdz(z,p) is its derivative function handle.
        
        y is the actual measurement value, subject to measurement noise.
        
        p is the positions of the robots.
        
        z_neighbor is the list of z estimations collected from the neighbors(including self).
        """
        A = self.dfdz(self._zbar)
        C = dhdz(self.z,p)
        R = self.R_mag*np.eye(len(y))
        Q = self.Q_mag*np.eye(len(self.z))
        
        # The Kalman Gain
        K = A.dot(self.P).dot(C.T).dot(    np.linalg.inv(C.dot(self.P).dot(C.T)+R)      )

        # Mean and covariance update
        N = len(z_neighbor)
        
        self._zbar= self.z+K.dot(y-h(self.z,p)) +\
                    self.C_gain*np.ones((1,N)).dot(z_neighbor-self.z).flatten() # The consensus term.
        
        self.z = self.f(self._zbar)
        
        self.P = A.dot(self.P).dot(A.T)+ Q- K.dot(C.dot(self.P).dot(C.T)+R).dot(K.T)
        
    def update_and_estimate_loc(self,h,dhdz,y,p,z_neighbor):
        if not np.any(y == np.inf):
            self.update(h,dhdz,y,p,z_neighbor)

        return self.z[:len(self.q)]

    
        
        

In [15]:
def circulant(i,q,p,prev,post,undirected=False):
    """
        Generate a circulant graph with len(p) nodes, node i connected with [i-prev:i+post],i-prev and i+post included but self-loop eliminated.
    """
    n = len(p)
    G = nx.DiGraph()
    edges = [(j%n,i) for i in range(n) for j in range(i-prev,i+post+1)]
    G.add_edges_from(edges)
    G.remove_edges_from(nx.selfloop_edges(G))
    if undirected:
        G = G.to_undirected()
    return G
    

In [16]:
def main(initial_locs,comm_network_generator,N_iter,C_gain):
    """
    Note: 
    comm_network_generator(i,qs,ps) is the expected signature of the second argument.

        Given the iteration number i, source location qs, sensor location ps,
        comm_network_generator return the current communication network G.

    """
    # Set up mobile sensors and sources
    sensor_names = ["mobile_sensor_{}".format(i+1) for i in range(len(initial_locs))]
    mobile_sensors = [Robot(loc,name) for loc,name in zip(initial_locs,sensor_names)]

    src_locs = [[8.,8]]
    src_names = ["source_{}".format(i+1) for i in range(len(src_locs))]
    source = [Robot(loc,name) for loc,name in zip(src_locs,src_names)]

    # Set up virtual sensors
    C1=-0.1 # Setting C1 as a negative number mitigates the blowing-up effect when the sensors are close to the source.
    C0=0
    k=1
    b=-2
    std = 0.1
    vs = virtual_sensor(C1,C0,b,k,std)

    # Set up location estimator
    n_sen = len(mobile_sensors)
    n_src = len(source)

    C1s=C1*np.ones(n_sen)
    C0s = C0*np.ones(n_sen)
    ks = k * np.ones(n_sen)
    bs = b*np.ones(n_sen)

    q_0 = np.random.rand(len(mobile_sensors),2)*10
    estimators = [ConsensusEKF(q,C_gain=C_gain) for q in q_0]

    # Set up data logger
    log = logger(sensor_names,src_names)
    log.est_locs_log =dict({"Est {}".format(sensor):[] for sensor in sensor_names})
    
    # Assume fixed topology, pre-set the measurement model and waypoint planner(expensive to be created in the main loop)
    ps = np.array([r.loc for r in mobile_sensors])
    qs = np.array([s.loc for s in source])
    G = comm_network_generator(0,qs,ps)
   
    hs = []
    dhdzs = []
    f_dLdps = []
    for i in G.nodes():
        N_i = [i]+list(G[i])     
       
        C1s_i=C1s[N_i]
        C0s_i = C0s[N_i]
        ks_i = ks[N_i]
        bs_i = bs[N_i]
        h=partial(joint_meas_func,C1s_i,C0s_i,ks_i,bs_i)# Freeze the coefficients, the signature becomes h(z,ps)
        dhdzs.append(partial(analytic_dhdz,C1s_i,C0s_i,ks_i,bs_i))

        hs.append(h)
        
        f_dLdps.append(partial(analytic_dLdp,C1s=C1s_i,C0s=C0s_i,ks=ks_i,bs=bs_i))

    # Enter main loop
    for _ in range(N_iter):

        # Measure
        ps = np.array([r.loc for r in mobile_sensors])
        qs = np.array([s.loc for s in source])
        '''The zhats are newly added'''
        zhats = np.array([est.z for est in estimators])

        y = vs.measurement(qs,ps)

        # Set up the current communication network.
        G = comm_network_generator(1,qs,ps)
        log.comm_network.append(G)

        # Record
        for s in source:
            log.src_locs[s.name].append(s.loc)



        # Estimate, plan and move a step for each robot.
        for i in G.nodes():
            # Get all robots queriable in the comm. network, including i.
            # Note: i's place within N_i is always at index 0

            """ The following line gets all the immediate neighbors rather than all ancestors, which is way this Notebook is prefixed with P2P(point to point)"""

            N_i = [i]+list(G[i]) 

            n_sen = len(N_i)

            # Simulate query of information from other robots. 
            ps_i =ps[N_i]
            y_i = y[N_i]
            '''The gathering of z_neighbor is newly added'''
            z_neighbor = zhats[N_i]
            qhat_neighbor = z_neighbor[:,:2]

            # Estimate
            qhat = estimators[i].update_and_estimate_loc(hs[i],dhdzs[i],y_i,ps_i,z_neighbor)

            # Set up waypoint planner
            f_dLdp=f_dLdps[i]

            planning_timesteps = 1
            max_linear_speed = 0.22
            planning_dt = 1
            epsilon = 0.5
            # The FIM waypoint planning  
            waypoints=FIM_ascent_path_planning(f_dLdp,qhat_neighbor,ps_i,n_sen,\
                            planning_timesteps,\
                            max_linear_speed,\
                            planning_dt,\
                            epsilon)
            # Note: i's place within N_i is always at index 0
            # Move to the 0th waypoint
            mobile_sensors[i].update_loc(waypoints[0][0])
            m = mobile_sensors[i]

            log.sensor_locs[m.name].append(m.loc)
            log.est_locs_log["Est {}".format(m.name)].append(qhat)


    return log

In [18]:
in_deg = 1
for C_gain in [0,0.2]:
    initial_locs = np.random.rand(6,2)*2

    comm_network=lambda i,q,p:circulant(i,q,p,prev=in_deg,post=0,undirected=True)
    log = main(initial_locs,comm_network,100,C_gain)

    # Plotting
    record = log.export()

    filepath = "Consensus-gain{}-FIM-Mixture.pkl".format(C_gain)
    with open(filepath,'wb') as file:
        pkl.dump(record,file)